In [1]:
# === Шаг 1: Импорт необходимых библиотек ===
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
import os # Добавим для удобной работы с путями

# === Шаг 2: Настройка и загрузка модели (делается один раз) ===

# Проверяем доступность CUDA и устанавливаем устройство
if torch.cuda.is_available():
    device = "cuda"
    print(f"CUDA доступна! Используем GPU: {torch.cuda.get_device_name(0)}")
else:
    device = "cpu"
    print("CUDA недоступна, используется CPU. Это будет медленно.")

# Пути к модели
model_checkpoint_path = "/app/MODELS/sam_vit_h_4b8939.pth"
model_type = "vit_h"

# Загружаем модель в память
print("\nЗагрузка модели Segment Anything...")
sam = sam_model_registry[model_type](checkpoint=model_checkpoint_path)
sam.to(device=device)
print("Модель успешно загружена.")

# Инициализируем генератор масок
mask_generator = SamAutomaticMaskGenerator(sam)
print("Генератор автоматических масок готов.")

CUDA доступна! Используем GPU: NVIDIA GeForce RTX 3080 Ti

Загрузка модели Segment Anything...
Модель успешно загружена.
Генератор автоматических масок готов.


In [8]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import xml.etree.ElementTree as ET
from xml.dom import minidom
import zipfile
import shutil

# ==============================================================================
# --- ⚙️ КОНФИГУРАЦИЯ: ИЗМЕНЯЙТЕ ТОЛЬКО ЭТОТ РАЗДЕЛ -----------------------------
# ==============================================================================

# 1. Укажите имя файла вашей панорамы (включая расширение .jpg или .png)
PANORAMA_FILENAME = "1_normals.jpg"

# 2. Укажите папку, где лежат ваши исходные панорамы
BASE_INPUT_DIR = "Vistino20241014_E57"

# 3. Укажите название главной папки для всех результатов
MAIN_OUTPUT_DIR = "CVAT_Workspace"

# 4. Определите классы для кнопок
CLASS_NAMES = ["Фон", "Земля", "Человек", "Растительность", "Транспорт", "Конструкции", "Здание"]

# ==============================================================================
# --- 🤖 ЛОГИКА СКРИПТА: ЭТУ ЧАСТЬ МОЖНО НЕ ТРОГАТЬ -----------------------------
# ==============================================================================

class PanoramaProcessor:
    def __init__(self, panorama_filename, input_dir, output_dir, class_names):
        self.panorama_filename = panorama_filename
        self.panorama_base_name = os.path.splitext(panorama_filename)[0]
        self.input_dir = input_dir
        
        # --- Создание динамических путей ---
        self.pano_output_dir = os.path.join(output_dir, self.panorama_base_name)
        self.source_masks_dir = os.path.join(self.pano_output_dir, "1_generated_masks")
        self.classified_masks_dir = os.path.join(self.pano_output_dir, "2_classified_masks")
        self.final_mask_dir = os.path.join(self.pano_output_dir, "3_final_mask")
        self.labelme_dir = os.path.join(self.pano_output_dir, "4_labelme_xml")
        self.final_zip_dir = os.path.join(self.pano_output_dir, "5_upload_to_cvat")
        
        self.class_names = class_names
        self.class_mapping = {name: i+1 for i, name in enumerate(class_names)}
        
        self.original_panorama_path = os.path.join(self.input_dir, self.panorama_filename)
        self.original_panorama = None
        self.original_panorama_rgb = None
        
        # Переменные для интерактивного классификатора
        self.mask_files = []
        self.current_mask_index = 0
        self.output_area = widgets.Output()

    def _setup_directories(self):
        """Создает всю необходимую структуру папок."""
        print(f"🗂️ Создание рабочей папки: {self.pano_output_dir}")
        for path in [self.source_masks_dir, self.classified_masks_dir, self.final_mask_dir, self.labelme_dir, self.final_zip_dir]:
            os.makedirs(path, exist_ok=True)
            
    def _load_panorama(self):
        """Загружает исходную панораму."""
        if not os.path.exists(self.original_panorama_path):
            print(f"❌ ОШИБКА: Не могу найти панораму: {self.original_panorama_path}")
            return False
        self.original_panorama = cv2.imread(self.original_panorama_path)
        self.original_panorama_rgb = cv2.cvtColor(self.original_panorama, cv2.COLOR_BGR2RGB)
        print(f"✅ Панорама успешно найдена: {self.original_panorama_path}")
        return True

    def generate_masks(self, sam_mask_generator):
        """Шаг 1: Генерация масок с помощью SAM."""
        print("\n--- Шаг 1: Генерация масок с помощью SAM ---")
        print("Запускаю автоматическую сегментацию (это может занять время)...")
        masks = sam_mask_generator.generate(self.original_panorama_rgb)
        print(f"Готово! Найдено {len(masks)} масок.")

        print(f"Сохранение масок в папку: '{self.source_masks_dir}'...")
        for i, ann in enumerate(masks):
            image_mask = (ann['segmentation'] * 255).astype(np.uint8)
            cv2.imwrite(os.path.join(self.source_masks_dir, f"mask_{i}.png"), image_mask)
        print("✅ Все маски сохранены.")
        
    def classify_masks_interactive(self):
        """Шаг 2: Интерактивная классификация масок."""
        print("\n--- Шаг 2: Интерактивная классификация масок ---")
        self.mask_files = sorted([f for f in os.listdir(self.source_masks_dir) if f.endswith('.png')])
        if not self.mask_files:
            print("--- Нет масок для классификации. Пропускаем этот шаг. ---")
            return

        print("Просто нажимайте на кнопки с нужным классом.")
        
        # --- Создание виджетов ---
        buttons = [widgets.Button(description=name, button_style='primary') for name in self.class_names]
        buttons.extend([
            widgets.Button(description="Пропустить", button_style='info'),
            widgets.Button(description="Стоп", button_style='danger')
        ])
        button_box = widgets.HBox(buttons)
        
        for btn in buttons:
            btn.on_click(self._on_button_click)
            
        display(button_box, self.output_area)
        self._display_current_mask()

    def _display_current_mask(self):
        """Вспомогательная функция для отображения в виджете."""
        if self.current_mask_index >= len(self.mask_files):
            with self.output_area:
                clear_output()
                print("🎉 Все маски классифицированы! Переходим к следующему шагу.")
            self.combine_and_export() # Автоматический запуск следующего шага
            return
            
        filename = self.mask_files[self.current_mask_index]
        filepath = os.path.join(self.source_masks_dir, filename)
        mask_image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        
        overlay = np.zeros_like(self.original_panorama)
        overlay[mask_image > 0] = (0, 255, 0)
        highlighted = cv2.addWeighted(self.original_panorama, 1, overlay, 0.6, 0)
        
        with self.output_area:
            clear_output(wait=True)
            fig, ax = plt.subplots(1, 1, figsize=(15, 7))
            ax.imshow(cv2.cvtColor(highlighted, cv2.COLOR_BGR2RGB))
            ax.set_title(f"Классифицируем: {filename} | Осталось: {len(self.mask_files) - self.current_mask_index}")
            ax.axis('off')
            plt.show()

    def _on_button_click(self, button):
        """Обработчик нажатия кнопок."""
        action = button.description
        filename = self.mask_files[self.current_mask_index]
        source_path = os.path.join(self.source_masks_dir, filename)
        
        if action == "Стоп":
            with self.output_area:
                clear_output()
                print("Процесс остановлен. Запустите ячейку заново, чтобы продолжить.")
            return
            
        elif action != "Пропустить":
            clean_name = action.strip().replace(" ", "_")
            new_filename = f"{self.panorama_base_name}_{clean_name}_{self.current_mask_index}.png"
            dest_path = os.path.join(self.classified_masks_dir, new_filename)
            shutil.move(source_path, dest_path)
        
        self.current_mask_index += 1
        self._display_current_mask()
        
    def combine_and_export(self):
        """Шаги 3, 4, 5: Сборка маски, создание XML и архивация."""
        print("\n--- Шаг 3: Сборка финальной маски для CVAT ---")
        height, width, _ = self.original_panorama.shape
        final_mask = np.zeros((height, width), dtype=np.uint8)
        
        for filename in os.listdir(self.classified_masks_dir):
            try:
                class_name = filename.split('_')[2]
                if class_name in self.class_mapping:
                    class_id = self.class_mapping[class_name]
                    mask = cv2.imread(os.path.join(self.classified_masks_dir, filename), cv2.IMREAD_GRAYSCALE)
                    if mask is not None:
                        final_mask[mask > 0] = class_id
            except IndexError:
                continue
                
        final_mask_path = os.path.join(self.final_mask_dir, f"{self.panorama_base_name}_mask.png")
        cv2.imwrite(final_mask_path, final_mask)
        print(f"✅ Финальная маска сохранена: {final_mask_path}")

        # --- Шаг 4: Создание LabelMe XML ---
        print("\n--- Шаг 4: Создание LabelMe XML ---")
        annotation = ET.Element('annotation')
        ET.SubElement(annotation, 'filename').text = self.panorama_filename
        ET.SubElement(annotation, 'folder').text = ''
        imagesize = ET.SubElement(annotation, 'imagesize')
        ET.SubElement(imagesize, 'nrows').text = str(height)
        ET.SubElement(imagesize, 'ncols').text = str(width)
        
        object_counter = 0
        id_to_name = {v: k for k, v in self.class_mapping.items()}
        for class_id in np.unique(final_mask):
            if class_id == 0: continue
            
            binary_mask = np.uint8(final_mask == class_id) * 255
            contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            class_name = id_to_name.get(class_id, "unknown")
            
            for contour in contours:
                if cv2.contourArea(contour) < 5: continue
                obj = ET.SubElement(annotation, 'object')
                ET.SubElement(obj, 'name').text = class_name
                ET.SubElement(obj, 'deleted').text = '0'
                ET.SubElement(obj, 'id').text = str(object_counter)
                object_counter += 1
                polygon = ET.SubElement(obj, 'polygon')
                for point in contour.squeeze():
                    pt = ET.SubElement(polygon, 'pt')
                    ET.SubElement(pt, 'x').text = str(int(point[0]))
                    ET.SubElement(pt, 'y').text = str(int(point[1]))

        xml_string = ET.tostring(annotation, 'utf-8')
        reparsed = minidom.parseString(xml_string)
        pretty_xml = '\n'.join(reparsed.toprettyxml(indent="  ").split('\n')[1:])
        final_xml = '<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n' + pretty_xml
        
        xml_filepath = os.path.join(self.labelme_dir, f"{self.panorama_base_name}.xml")
        with open(xml_filepath, 'w', encoding='utf-8') as f:
            f.write(final_xml)
        print(f"✅ XML-аннотация сохранена: {xml_filepath}")
        
        # --- Шаг 5: Создание ZIP-архива ---
        print("\n--- Шаг 5: Создание ZIP-архива ---")
        zip_filepath = os.path.join(self.final_zip_dir, f"{self.panorama_base_name}_for_cvat.zip")
        with zipfile.ZipFile(zip_filepath, 'w') as zipf:
            zipf.write(self.original_panorama_path, arcname=self.panorama_filename)
            zipf.write(xml_filepath, arcname=f"{self.panorama_base_name}.xml")
        print("\n----------------------------------------------------")
        print(f"🎉 ПОБЕДА! Финальный ZIP-архив готов для загрузки в CVAT:")
        print(f"➡️  {zip_filepath}")


# --- Главный блок выполнения ---
# Убедитесь, что переменная `mask_generator` создана из предыдущей ячейки
if 'mask_generator' in locals():
    processor = PanoramaProcessor(PANORAMA_FILENAME, BASE_INPUT_DIR, MAIN_OUTPUT_DIR, CLASS_NAMES)
    processor._setup_directories()
    if processor._load_panorama():
        # Решите, что делать: генерировать маски заново или использовать существующие
        # Если папка с сгенерированными масками пуста, генерируем их
        if not os.listdir(processor.source_masks_dir):
             processor.generate_masks(mask_generator)
        
        # Если есть неклассифицированные маски, запускаем виджет
        if os.listdir(processor.source_masks_dir):
            processor.classify_masks_interactive()
        # Иначе, если есть уже классифицированные маски, сразу переходим к экспорту
        elif os.listdir(processor.classified_masks_dir):
            print("\nНайдены уже классифицированные маски. Переходим к экспорту.")
            processor.combine_and_export()
        else:
            print("\nНе найдено ни сгенерированных, ни классифицированных масок для обработки.")
else:
    print("❌ ОШИБКА: Сначала запустите ячейку с инициализацией `mask_generator`.")

🗂️ Создание рабочей папки: CVAT_Workspace/1_normals
✅ Панорама успешно найдена: Vistino20241014_E57/1_normals.jpg

--- Шаг 1: Генерация масок с помощью SAM ---
Запускаю автоматическую сегментацию (это может занять время)...
Готово! Найдено 35 масок.
Сохранение масок в папку: 'CVAT_Workspace/1_normals/1_generated_masks'...
✅ Все маски сохранены.

--- Шаг 2: Интерактивная классификация масок ---
Просто нажимайте на кнопки с нужным классом.


Output()

In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import xml.etree.ElementTree as ET
from xml.dom import minidom
import zipfile
import shutil

# ==============================================================================
# --- ⚙️ КОНФИГУРАЦИЯ: ИЗМЕНЯЙТЕ ТОЛЬКО ЭТОТ РАЗДЕЛ -----------------------------
# ==============================================================================

# 1. Укажите имя файла вашей панорамы (включая расширение .jpg или .png)
PANORAMA_FILENAME = "1_normals.jpg"

# 2. Укажите папку, где лежат ваши исходные панорамы
BASE_INPUT_DIR = "Vistino20241014_E57"

# 3. Укажите название главной папки для всех результатов
MAIN_OUTPUT_DIR = "CVAT_Workspace"

# 4. Определите классы для кнопок
CLASS_NAMES = ["Фон", "Земля", "Человек", "Растительность", "Транспорт", "Конструкции", "Здание"]

# ==============================================================================
# --- 🤖 ЛОГИКА СКРИПТА (Ячейка 1) -----------------------------------------------
# ==============================================================================

class PanoramaProcessor:
    def __init__(self, panorama_filename, input_dir, output_dir, class_names):
        # ... (эта часть без изменений) ...
        self.panorama_filename = panorama_filename
        self.panorama_base_name = os.path.splitext(panorama_filename)[0]
        self.input_dir = input_dir
        
        # --- Создание динамических путей ---
        self.pano_output_dir = os.path.join(output_dir, self.panorama_base_name)
        self.source_masks_dir = os.path.join(self.pano_output_dir, "1_generated_masks")
        self.classified_masks_dir = os.path.join(self.pano_output_dir, "2_classified_masks")
        self.final_mask_dir = os.path.join(self.pano_output_dir, "3_final_mask")
        self.labelme_dir = os.path.join(self.pano_output_dir, "4_labelme_xml")
        self.final_zip_dir = os.path.join(self.pano_output_dir, "5_upload_to_cvat")
        
        self.class_names = class_names
        self.class_mapping = {name: i + 1 for i, name in enumerate(class_names)}
        
        self.original_panorama_path = os.path.join(self.input_dir, self.panorama_filename)
        self.original_panorama = None
        self.original_panorama_rgb = None
        
        self.mask_files = []
        self.current_mask_index = 0
        self.output_area = widgets.Output()

    def setup_and_load(self):
        """Подготовка папок и загрузка панорамы."""
        print(f"🗂️ Настройка рабочей папки: {self.pano_output_dir}")
        for path in [self.source_masks_dir, self.classified_masks_dir, self.final_mask_dir, self.labelme_dir, self.final_zip_dir]:
            os.makedirs(path, exist_ok=True)
            
        if not os.path.exists(self.original_panorama_path):
            print(f"❌ ОШИБКА: Панорама не найдена: {self.original_panorama_path}")
            return False
        self.original_panorama = cv2.imread(self.original_panorama_path)
        self.original_panorama_rgb = cv2.cvtColor(self.original_panorama, cv2.COLOR_BGR2RGB)
        print(f"✅ Панорама успешно найдена: {self.original_panorama_path}")
        return True

    def generate_masks(self, sam_mask_generator):
        """Шаг 1: Генерация масок с помощью SAM."""
        print("\n--- Шаг 1: Генерация масок ---")
        if os.listdir(self.source_masks_dir):
            print("Маски уже сгенерированы. Пропускаем этот шаг.")
            return

        print("Запускаю автоматическую сегментацию (это может занять время)...")
        masks = sam_mask_generator.generate(self.original_panorama_rgb)
        print(f"Готово! Найдено {len(masks)} масок.")

        print(f"Сохранение масок в '{self.source_masks_dir}'...")
        for i, ann in enumerate(masks):
            image_mask = (ann['segmentation'] * 255).astype(np.uint8)
            cv2.imwrite(os.path.join(self.source_masks_dir, f"mask_{i}.png"), image_mask)
        print("✅ Все маски сохранены.")
        
    def classify_masks_interactive(self):
        """Шаг 2: Интерактивная классификация масок."""
        print("\n--- Шаг 2: Интерактивная классификация ---")
        self.mask_files = sorted([f for f in os.listdir(self.source_masks_dir) if f.endswith('.png')])
        if not self.mask_files:
            print("Нет масок для классификации. Можно запускать следующую ячейку.")
            return
            
        print("Нажимайте на кнопки для присвоения класса.")
        
        buttons = [widgets.Button(description=name, button_style='primary') for name in self.class_names]
        buttons.extend([widgets.Button(description="Пропустить", button_style='info'), widgets.Button(description="Стоп", button_style='danger')])
        button_box = widgets.HBox(buttons)
        
        for btn in buttons: btn.on_click(self._on_button_click)
        
        display(button_box, self.output_area)
        self._display_current_mask()

    def _display_current_mask(self):
        """Вспомогательная функция для отображения в виджете."""
        if self.current_mask_index >= len(self.mask_files):
            with self.output_area:
                clear_output()
                print("🎉 Все маски классифицированы! Теперь можно запустить следующую ячейку. 🎉")
            return
            
        filename = self.mask_files[self.current_mask_index]
        filepath = os.path.join(self.source_masks_dir, filename)
        mask_image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        
        overlay = np.zeros_like(self.original_panorama)
        overlay[mask_image > 0] = (0, 255, 0)
        highlighted = cv2.addWeighted(self.original_panorama, 1, overlay, 0.6, 0)
        
        with self.output_area:
            clear_output(wait=True)
            fig, ax = plt.subplots(1, 1, figsize=(15, 7))
            ax.imshow(cv2.cvtColor(highlighted, cv2.COLOR_BGR2RGB))
            ax.set_title(f"Классифицируем: {filename} | Осталось: {len(self.mask_files) - self.current_mask_index}")
            ax.axis('off')
            plt.show()

    def _on_button_click(self, button):
        """Обработчик нажатия кнопок."""
        action = button.description
        
        if action == "Стоп":
            with self.output_area:
                clear_output()
                print("🚦 Процесс остановлен. Запустите следующую ячейку для экспорта уже размеченных масок.")
            return
            
        filename = self.mask_files[self.current_mask_index]
        source_path = os.path.join(self.source_masks_dir, filename)
        
        if action != "Пропустить" and os.path.exists(source_path):
            clean_name = action.strip().replace(" ", "_")
            new_filename = f"{self.panorama_base_name}_{clean_name}_{self.current_mask_index}.png"
            dest_path = os.path.join(self.classified_masks_dir, new_filename)
            shutil.move(source_path, dest_path)
        
        self.current_mask_index += 1
        self._display_current_mask()
        
    def create_final_dataset(self):
        """Сборка маски, создание XML и архивация."""
        print("\n--- Шаг 3: Сборка и экспорт финального датасета ---")
        if not os.listdir(self.classified_masks_dir):
            print("❌ Нет классифицированных масок для экспорта.")
            return

        # ... (эта часть без изменений) ...
        height, width, _ = self.original_panorama.shape
        final_mask = np.zeros((height, width), dtype=np.uint8)
        for filename in os.listdir(self.classified_masks_dir):
            try:
                class_name = filename.split('_')[2]
                if class_name in self.class_mapping:
                    class_id = self.class_mapping[class_name]
                    mask = cv2.imread(os.path.join(self.classified_masks_dir, filename), cv2.IMREAD_GRAYSCALE)
                    if mask is not None: final_mask[mask > 0] = class_id
            except IndexError: continue
        final_mask_path = os.path.join(self.final_mask_dir, f"{self.panorama_base_name}_mask.png")
        cv2.imwrite(final_mask_path, final_mask)
        print(f"✅ Финальная маска сохранена: {final_mask_path}")

        # --- Создание XML ---
        annotation = ET.Element('annotation')
        ET.SubElement(annotation, 'filename').text = self.panorama_filename
        ET.SubElement(annotation, 'folder').text = ''
        imagesize = ET.SubElement(annotation, 'imagesize')
        ET.SubElement(imagesize, 'nrows').text = str(height)
        ET.SubElement(imagesize, 'ncols').text = str(width)
        object_counter = 0
        id_to_name = {v: k for k, v in self.class_mapping.items()}
        for class_id in np.unique(final_mask):
            if class_id == 0: continue
            binary_mask = np.uint8(final_mask == class_id) * 255
            contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            class_name = id_to_name.get(class_id, "unknown")
            for contour in contours:
                if cv2.contourArea(contour) < 5: continue
                obj = ET.SubElement(annotation, 'object')
                ET.SubElement(obj, 'name').text = class_name
                ET.SubElement(obj, 'deleted').text = '0'
                ET.SubElement(obj, 'id').text = str(object_counter)
                object_counter += 1
                polygon = ET.SubElement(obj, 'polygon')
                for point in contour.squeeze():
                    pt = ET.SubElement(polygon, 'pt')
                    ET.SubElement(pt, 'x').text = str(int(point[0]))
                    ET.SubElement(pt, 'y').text = str(int(point[1]))
        xml_string = ET.tostring(annotation, 'utf-8')
        reparsed = minidom.parseString(xml_string)
        pretty_xml = '\n'.join(reparsed.toprettyxml(indent="  ").split('\n')[1:])
        final_xml = '<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n' + pretty_xml
        xml_filepath = os.path.join(self.labelme_dir, f"{self.panorama_base_name}.xml")
        with open(xml_filepath, 'w', encoding='utf-8') as f: f.write(final_xml)
        print(f"✅ XML-аннотация сохранена: {xml_filepath}")
        
        # --- Создание ZIP-архива ---
        zip_filepath = os.path.join(self.final_zip_dir, f"{self.panorama_base_name}_for_cvat.zip")
        with zipfile.ZipFile(zip_filepath, 'w') as zipf:
            zipf.write(self.original_panorama_path, arcname=self.panorama_filename)
            zipf.write(xml_filepath, arcname=f"{self.panorama_base_name}.xml")
        print("\n----------------------------------------------------")
        print(f"🎉 ПОБЕДА! Финальный ZIP-архив готов для загрузки в CVAT:")
        print(f"➡️  {zip_filepath}")

# --- Главный блок выполнения (Ячейка 1) ---
if 'mask_generator' in locals():
    processor = PanoramaProcessor(PANORAMA_FILENAME, BASE_INPUT_DIR, MAIN_OUTPUT_DIR, CLASS_NAMES)
    if processor.setup_and_load():
        processor.generate_masks(mask_generator)
        processor.classify_masks_interactive()
else:
    print("❌ ОШИБКА: Сначала запустите ячейку с инициализацией `mask_generator`.")

🗂️ Настройка рабочей папки: CVAT_Workspace/1_normals
✅ Панорама успешно найдена: Vistino20241014_E57/1_normals.jpg

--- Шаг 1: Генерация масок ---
Запускаю автоматическую сегментацию (это может занять время)...
Готово! Найдено 35 масок.
Сохранение масок в 'CVAT_Workspace/1_normals/1_generated_masks'...
✅ Все маски сохранены.

--- Шаг 2: Интерактивная классификация ---
Нажимайте на кнопки для присвоения класса.


Output()

In [3]:
# --- Главный блок выполнения (Ячейка 2) ---
# Запустите эту ячейку после того, как закончите классификацию в виджете выше.

if 'processor' in locals() and os.path.isdir(processor.classified_masks_dir):
    # Проверяем, есть ли что экспортировать
    if os.listdir(processor.classified_masks_dir):
        # Если есть, вызываем метод для создания датасета
        processor.create_final_dataset()
    else:
        print("🟡 Папка с классифицированными масками пуста. Нечего экспортировать.")
        print("Возможно, вы пропустили все маски или нажали 'Стоп' до классификации.")
else:
    print("❌ ОШИБКА: Объект 'processor' не найден.")
    print("Пожалуйста, сначала успешно запустите первую ячейку.")


--- Шаг 3: Сборка и экспорт финального датасета ---
✅ Финальная маска сохранена: CVAT_Workspace/1_normals/3_final_mask/1_normals_mask.png
✅ XML-аннотация сохранена: CVAT_Workspace/1_normals/4_labelme_xml/1_normals.xml

----------------------------------------------------
🎉 ПОБЕДА! Финальный ZIP-архив готов для загрузки в CVAT:
➡️  CVAT_Workspace/1_normals/5_upload_to_cvat/1_normals_for_cvat.zip
